<a href="https://colab.research.google.com/github/Layaa-V/Customer-Segmentation-Analysis/blob/main/Customer_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raw Data Info

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('marketing_campaign.csv', sep="\t")

In [ ]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [ ]:
df.isnull().sum()

,0
ID,0
Year_Birth,0
Education,0
Marital_Status,0
Income,24
Kidhome,0
Teenhome,0
Dt_Customer,0
Recency,0
MntWines,0


Therefore we see that only the column 'income' has null entries.

In [ ]:
df.duplicated().sum()

np.int64(0)

There are no duplicate rows in dataset.

# Data Cleaning

In [ ]:
df_new = df.copy()

In [ ]:
# Since only the column 'income' has null values, hence imputing data only for that column
# filling null spaces using mean
df_new['Income'].fillna(df_new['Income'].median(), inplace = True)

# printing to verify
print(df_new['Income'].isnull().sum())

0


In [ ]:
df_new.isnull().sum().sum()
# Hence, there are no null values in the dataset now.

np.int64(0)



*   We see that 'Education' and 'Marital_Status' are categorical columns. Therefore, finding their unique categories and reducing to broader classes if possible.




In [ ]:
df['Education'].unique()

array(['Graduation', 'PhD', 'Master', 'Basic', '2n Cycle'], dtype=object)

In [ ]:
df['Marital_Status'].unique()

array(['Single', 'Together', 'Married', 'Divorced', 'Widow', 'Alone',
       'Absurd', 'YOLO'], dtype=object)

In [ ]:
education_mapping = {'Basic': 'Undergraduate','2n Cycle': 'Graduate', 'Graduation': 'Graduate','Graduation': 'Graduate',
            'Master': 'Postgraduate','PhD': 'Postgraduate'}
df_new['Education'] = df_new['Education'].replace(education_mapping)

In [ ]:
marital_mapping = {'Married': 'Partnered','Together': 'Partnered','Single': 'Single','Divorced': 'Single',
                   'Widow': 'Single','Alone': 'Single','Absurd': 'Single','YOLO': 'Single'}
df_new['Marital_Status'] = df_new['Marital_Status'].replace(marital_mapping)

In [ ]:
# New categories
df_new['Education'].unique()

array(['Graduate', 'Postgraduate', 'Undergraduate'], dtype=object)

In [ ]:
# New categories
df_new['Marital_Status'].unique()

array(['Single', 'Partnered'], dtype=object)



*   Making conclusive columns for repetitive entities (amount spent, campaigns)




In [ ]:
 # Total_Spending column (sum of all spending categories)
spending_cols = [col for col in df_new.columns if 'Mnt' in col]
if spending_cols:
    df_new['Total_Spending'] = df_new[spending_cols].sum(axis=1)

In [ ]:
df_new['Total_Spending'].head()

,Total_Spending
0,1617
1,27
2,776
3,53
4,422


In [ ]:
campaign_cols = [col for col in df_new.columns if 'Cmp' in col or 'AcceptedCmp' in col]
if campaign_cols:
  df_new['Total_Campaigns_Accepted'] = df_new[campaign_cols].sum(axis=1)

In [ ]:
df_new['Total_Campaigns_Accepted'].head()

,Total_Campaigns_Accepted
0,0
1,0
2,0
3,0
4,0




*   Concluding Family Details




In [ ]:
df_new['Total_Children'] = df_new['Kidhome'] + df_new['Teenhome']
df_new['Has_Children'] = (df_new['Total_Children'] > 0).astype(int)

In [ ]:
df_new[['Total_Children','Has_Children']].head()

,Total_Children,Has_Children
0,0,0
1,2,1
2,0,0
3,1,1
4,1,1




*   Removing customer ID




In [ ]:
df_new = df_new.drop(columns='ID')



*   Extracting date/time info from date objects





In [ ]:
df_new['Dt_Customer'] = pd.to_datetime(df_new['Dt_Customer'], errors='coerce')

# checking for null values
nan_count_before = df_new['Dt_Customer'].isnull().sum()
if nan_count_before > 0:
  median_date = df_new['Dt_Customer'].median()
  df_new['Dt_Customer'].fillna(median_date, inplace=True)

df_new['Enrollment_Month'] = df_new['Dt_Customer'].dt.month
df_new['Enrollment_Year'] = df_new['Dt_Customer'].dt.year

In [ ]:
df_new.drop(columns='Dt_Customer', inplace=True)

In [ ]:
# new dataset shape
df_new.shape

(2240, 33)

# Encoding Categorical Data

In [ ]:
categorical_cols = df_new.select_dtypes(include=['object']).columns
print(categorical_cols)

Index(['Education', 'Marital_Status'], dtype='object')


In [ ]:
df_new = pd.get_dummies(df_new, columns=categorical_cols, drop_first=True)
print('New shape of dataset:', df_new.shape)

New shape of dataset: (2240, 34)


In [ ]:
# new columns
df_new.columns

Index(['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines',
       'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response',
       'Total_Spending', 'Total_Campaigns_Accepted', 'Total_Children',
       'Has_Children', 'Enrollment_Month', 'Enrollment_Year',
       'Education_Postgraduate', 'Education_Undergraduate',
       'Marital_Status_Single'],
      dtype='object')

In [ ]:
df_new.to_csv('cdata.csv', index=False)

# Dimensionality Reduction

In [ ]:
# scaling the features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_new)

In [ ]:
pca_temp = PCA()
pca_temp.fit(df_scaled)
cumsum_var = np.cumsum(pca_temp.explained_variance_ratio_)
n_components = np.argmax(cumsum_var >= 0.95) + 1
print(f"Number of components for 95% variance: {n_components}")

Number of components for 95% variance: 24


In [ ]:
pca = PCA(n_components=n_components)
df_pca = pca.fit_transform(df_scaled)

In [ ]:
pca_columns = [f'PC{i+1}' for i in range(n_components)]
df_final = pd.DataFrame(df_pca, columns=pca_columns)

In [ ]:
df_final.shape

(2240, 24)

In [ ]:
df_final.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24
0,4.440072,-0.441154,-1.212194,-2.921589,1.702014,-0.933004,1.204538,-0.908896,0.584208,1.214454,...,1.333772,-0.299663,-0.151714,-0.972105,-0.239929,0.422676,-0.889899,-0.199140,-1.303815,-0.180862
1,-2.963183,0.130282,-0.043969,1.508977,-0.399798,1.895560,0.422089,1.633550,1.253528,0.663276,...,0.497868,-0.743200,-0.008859,-0.445665,-0.074259,-0.454633,-0.287264,-0.101832,-0.248411,-0.013024
2,2.041758,-1.179137,-1.362415,0.143487,0.277441,-0.888433,-0.274979,0.218121,-0.220829,-0.712737,...,0.002531,0.258796,-0.252426,-0.231615,-0.465430,-0.557716,-0.746706,-0.298896,1.099553,-0.284858
3,-2.699342,-1.587484,0.451775,0.307774,-0.899329,0.957434,-1.705891,0.623604,0.450687,-0.382943,...,0.401572,-0.454179,-0.213320,-0.409084,-0.136673,-0.008345,0.090320,-0.239212,0.048614,-0.406167
4,-0.752083,0.122107,-0.662017,-0.228723,-0.716655,0.026204,-1.004335,-1.656556,-0.707655,0.603419,...,0.674702,0.486841,-0.276584,-0.213139,0.097726,0.258561,0.266930,0.439670,0.753134,0.462823


# Cleaned Data Output File

In [ ]:
df_final.to_csv('cleaned_customer_data.csv', index=False)